In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import os 
import seaborn as sns 
import sklearn as skl
import scipy.stats as stats
from scipy.stats import skew
from imblearn.over_sampling import SMOTE
from scipy.stats import norm 
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler 
import warnings  
warnings.filterwarnings('ignore') 
%matplotlib inline

In [2]:
df_continuous = pd.read_csv('Data/df_continuous.csv')
df_discrete = pd.read_csv('Data/df_discrete.csv')
df_categorical = pd.read_csv('Data/df_categorical.csv')

#Concatenating the Continuous and Discrete dataframes to create a Numerical Dataframe
df_numerical = pd.concat([df_continuous, df_discrete], axis =1 )

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
df_continuous.head()

,Income,Total Claim Amount,Customer Lifetime Value
0,56274,384.811147,2763.519279
1,0,1131.464935,6979.535903
2,48767,566.472247,12887.431650
3,0,529.881344,7645.861827
4,43836,138.130879,2813.692575


In [4]:
df_discrete.head()

,Months Since Policy Inception,Months Since Last Claim,Number of Policies,Monthly Premium Auto,Number of Open Complaints
0,5,32,1,69,0
1,42,13,8,94,0
2,38,18,2,108,0
3,65,18,7,106,0
4,44,12,1,73,0


In [5]:
df_numerical.head()

,Income,Total Claim Amount,Customer Lifetime Value,Months Since Policy Inception,Months Since Last Claim,Number of Policies,Monthly Premium Auto,Number of Open Complaints
0,56274,384.811147,2763.519279,5,32,1,69,0
1,0,1131.464935,6979.535903,42,13,8,94,0
2,48767,566.472247,12887.431650,38,18,2,108,0
3,0,529.881344,7645.861827,65,18,7,106,0
4,43836,138.130879,2813.692575,44,12,1,73,0


In [6]:
df_categorical.head()

,Education,Policy Type,Sales Channel,State,Vehicle Class,Response,EmploymentStatus,Marital Status,Coverage,Policy,Vehicle Size,Renew Offer Type,Location Code,Gender,Effective To Date,Expiry_date,Expiry_year,Expiry_month,Expiry_week_in_year,Expiry_day_of_week,Expiry_day_of_month
0,Bachelor,Corporate Auto,Agent,Washington,Two-Door Car,No,Employed,Married,Basic,Corporate L3,Medsize,Offer1,Suburban,F,2/24/11,2011-02-24,2011,2,8,3,24
1,Bachelor,Personal Auto,Agent,Arizona,Four-Door Car,No,Unemployed,Single,Extended,Personal L3,Medsize,Offer3,Suburban,F,1/31/11,2011-01-31,2011,1,5,0,31
2,Bachelor,Personal Auto,Agent,Nevada,Two-Door Car,No,Employed,Married,Premium,Personal L3,Medsize,Offer1,Suburban,F,2/19/11,2011-02-19,2011,2,7,5,19
3,Bachelor,Corporate Auto,Call Center,California,SUV,No,Unemployed,Married,Basic,Corporate L2,Medsize,Offer1,Suburban,M,1/20/11,2011-01-20,2011,1,3,3,20
4,Bachelor,Personal Auto,Agent,Washington,Four-Door Car,No,Employed,Single,Basic,Personal L1,Medsize,Offer1,Rural,M,2/3/11,2011-02-03,2011,2,5,3,3


In [7]:
# Helper function for skewness:

#To find the skewness of all the transoformations of the feature
def skew_features(feature):
    
    #Original Feature
    print('Original - Skewness: {}'.format(skew(df_numerical[feature])))

    # Log Transformation:
    df_numerical['Log_{}'.format(feature)]=np.log(df_numerical[feature]+1)
    print('Log Transformation - Skewness: {}'.format(skew(df_numerical['Log_{}'.format(feature)])))

    # Reciprocal Transformation:
    df_numerical['Reciprocal_{}'.format(feature)] = 1/(df_numerical[feature]+1)
    print('Reciprocal Transformation - Skewness: {}'.format(skew(df_numerical['Reciprocal_{}'.format(feature)])))

    # Square root Transformation:
    df_numerical['Sqrt_{}'.format(feature)] = df_numerical[feature]**(1/2)
    print('Square root Transformation - Skewness: {}'.format(skew(df_numerical['Sqrt_{}'.format(feature)])))

    # Exponential Transformation:
    df_numerical['Exponential_{}'.format(feature)] = df_numerical[feature]**(1/5)
    print('Exponential Transformation - Skewness: {}'.format(skew(df_numerical['Exponential_{}'.format(feature)])))

    #Boxcox Transormation
    df_numerical['Boxcox_{}'.format(feature)], param = stats.boxcox(df_numerical.Income+1) 
    print('Boxcox Transormation - Skewness: {}'.format(skew(df_numerical['Boxcox_{}'.format(feature)])))


## We will now take care of outliers in the numerial columns by squeezing them using log transforms and creating new features.

## We use log tranforms so that our data checks the multivariate normality assumption, which states that the numerical features should be normally distributed, for linear regression

### 1. Income 

In [8]:
# Values in the 'Income' column which are equal to zero
filt_1 = (df_numerical['Income'] == 0)
print('{:.2f}% of the "Income" feature are equal to zero'.format(df_numerical['Income'][filt_1].count()*100/df_numerical.shape[0]))
print('This is because Unemployed people have not disclosed any income.')

25.37% of the "Income" feature are equal to zero
This is because Unemployed people have not disclosed any income.


In [9]:
skew_features('Income')

Original - Skewness: 0.28684016057807576
Log Transformation - Skewness: -1.0979720542732008
Reciprocal Transformation - Skewness: 1.1322774247749843
Square root Transformation - Skewness: -0.4859544104288794
Exponential Transformation - Skewness: -0.9902263661832031
Boxcox Transormation - Skewness: -0.8559935285382285


### As the skewness closest to zero is only given by the Actual 'Income' feature, the other transforms are considered to be redundant and are removed.

In [10]:
df_numerical = df_numerical.drop(['Log_Income', 'Reciprocal_Income',
       'Sqrt_Income', 'Exponential_Income', 'Boxcox_Income'], axis=1)

### 2. Total Claim Amount 

In [11]:
skew_features('Total Claim Amount')

Original - Skewness: 1.7146841624611302
Log Transformation - Skewness: -1.7410834060353928
Reciprocal Transformation - Skewness: 16.58714957578826
Square root Transformation - Skewness: 0.13720872853139376
Exponential Transformation - Skewness: -0.9163720208877641
Boxcox Transormation - Skewness: -0.8559935285382285


### As the skewness is closest to zero for the Square root transformation, we keep that transformation and drop the others.

In [12]:
df_numerical = df_numerical.drop(['Log_Total Claim Amount',
       'Reciprocal_Total Claim Amount',
       'Exponential_Total Claim Amount', 'Boxcox_Total Claim Amount'], axis=1)

### 3. Customer Lifetime Value

In [13]:
skew_features('Customer Lifetime Value')

Original - Skewness: 3.031782271327885
Log Transformation - Skewness: 0.5760771725806841
Reciprocal Transformation - Skewness: 0.6950248312087297
Square root Transformation - Skewness: 1.5885907492952607
Exponential Transformation - Skewness: 0.9416386270217649
Boxcox Transormation - Skewness: -0.8559935285382285


### As the skewness is closest to zero for the Log transformation, we keep that transformation and drop the others.

In [14]:
df_numerical = df_numerical.drop(['Reciprocal_Customer Lifetime Value',
       'Sqrt_Customer Lifetime Value', 'Exponential_Customer Lifetime Value',
       'Boxcox_Customer Lifetime Value'], axis=1)

### 4. Months Since Policy Inception

In [15]:
skew_features('Months Since Policy Inception')

Original - Skewness: 0.040158365343045704
Log Transformation - Skewness: -1.528101041701703
Reciprocal Transformation - Skewness: 6.236979232119047
Square root Transformation - Skewness: -0.5943070084775823
Exponential Transformation - Skewness: -1.9422246017616085
Boxcox Transormation - Skewness: -0.8559935285382285


### As the skewness closest to zero is only given by the Original feature, the other transforms are considered to be redundant and are removed.

In [16]:
df_numerical = df_numerical.drop(['Log_Months Since Policy Inception',
       'Reciprocal_Months Since Policy Inception',
       'Sqrt_Months Since Policy Inception',
       'Exponential_Months Since Policy Inception',
       'Boxcox_Months Since Policy Inception'], axis=1)

### 5. Months Since Last Claim

In [17]:
skew_features('Months Since Last Claim')

Original - Skewness: 0.2785405598366762
Log Transformation - Skewness: -1.0071692983711606
Reciprocal Transformation - Skewness: 3.230872514233826
Square root Transformation - Skewness: -0.44347123859796145
Exponential Transformation - Skewness: -2.1443770434469354
Boxcox Transormation - Skewness: -0.8559935285382285


### As the skewness closest to zero is only given by the Original feature, the other transforms are considered to be redundant and are removed.

In [18]:
df_numerical = df_numerical.drop(['Log_Months Since Last Claim',
       'Reciprocal_Months Since Last Claim', 'Sqrt_Months Since Last Claim',
       'Exponential_Months Since Last Claim',
       'Boxcox_Months Since Last Claim'], axis=1)

### 6. Number of Policies

In [19]:
skew_features('Number of Policies')

Original - Skewness: 1.2531269015899718
Log Transformation - Skewness: 0.6575250625896181
Reciprocal Transformation - Skewness: -0.11332808236001006
Square root Transformation - Skewness: 0.8567039714262964
Exponential Transformation - Skewness: 0.6015690572516057
Boxcox Transormation - Skewness: -0.8559935285382285


### As the skewness is closest to zero for the Exponential transformation, we keep that transformation and drop the others.

In [20]:
df_numerical = df_numerical.drop(['Log_Number of Policies',
       'Reciprocal_Number of Policies', 'Sqrt_Number of Policies',
        'Boxcox_Number of Policies'], axis=1)

### 7. Monthly Premium Auto

In [21]:
skew_features('Monthly Premium Auto')

Original - Skewness: 2.123197701864424
Log Transformation - Skewness: 1.0134963376912063
Reciprocal Transformation - Skewness: -0.3758574428232091
Square root Transformation - Skewness: 1.493700416881424
Exponential Transformation - Skewness: 1.1850174745666269
Boxcox Transormation - Skewness: -0.8559935285382285


### As the skewness is closest to zero for the Reciprocal transformation, we keep that transformation and drop the others.

In [22]:
df_numerical = df_numerical.drop(['Log_Monthly Premium Auto',
       'Sqrt_Monthly Premium Auto', 'Exponential_Monthly Premium Auto',
       'Boxcox_Monthly Premium Auto'], axis=1)

### 8. Number of Open Complaints

In [23]:
skew_features('Number of Open Complaints')

Original - Skewness: 2.7828059596647505
Log Transformation - Skewness: 1.9986826526144468
Reciprocal Transformation - Skewness: -1.634656471505597
Square root Transformation - Skewness: 1.8390222712442665
Exponential Transformation - Skewness: 1.5133917779837824
Boxcox Transormation - Skewness: -0.8559935285382285


### As the skewness is closest to zero for the Boxcox transformation, we keep that transformation and drop the others.

In [24]:
df_numerical = df_numerical.drop(['Log_Number of Open Complaints',
       'Reciprocal_Number of Open Complaints',
       'Sqrt_Number of Open Complaints',
       'Exponential_Number of Open Complaints'], axis=1)

## We will Standardize the numerical data.

In [25]:
columns = df_numerical.columns
ss = StandardScaler()
df_numerical_ss = ss.fit_transform(df_numerical)
df_numerical_normalized = pd.DataFrame(df_numerical_ss, columns=columns)

In [26]:
#Attaching the Response column
df_categorical['Response'] = df_categorical['Response'].apply(lambda x: 0 if x=='No' else 1)
df_numerical_normalized = df_numerical_normalized.join(df_categorical['Response'])

## Now we remove redundant features of the date column except for the Expiry_month and Expiry_week_in_year features and generate dummy variables from the categorical data.

## Note: These features are only good for models being built for the year 2011 and if we are including any other year, the Expiry_year feature should also be included in the training process. 

In [27]:
df_cat = df_categorical.drop(['Effective To Date', 'Expiry_date', 'Expiry_year', 'Expiry_day_of_week', 'Expiry_day_of_month', 'Response'], axis=1)

cat_columns = df_cat.columns
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
for col in df_cat[cat_columns]:
    df_cat[col] = lb.fit_transform(df_cat[col])
    df_cat[col] = df_cat[col].astype('category')

# We drop one categorical encoding column from each feature to reduce the effect of multicollinearity.    
df_cat = pd.get_dummies(df_cat,drop_first=True)
df_cat.head()

,Education_1,Education_2,Education_3,Education_4,Policy Type_1,Policy Type_2,Sales Channel_1,Sales Channel_2,Sales Channel_3,State_1,State_2,State_3,State_4,Vehicle Class_1,Vehicle Class_2,Vehicle Class_3,Vehicle Class_4,Vehicle Class_5,EmploymentStatus_1,EmploymentStatus_2,EmploymentStatus_3,EmploymentStatus_4,Marital Status_1,Marital Status_2,Coverage_1,Coverage_2,Policy_1,Policy_2,Policy_3,Policy_4,Policy_5,Policy_6,Policy_7,Policy_8,Vehicle Size_1,Vehicle Size_2,Renew Offer Type_1,Renew Offer Type_2,Renew Offer Type_3,Location Code_1,Location Code_2,Gender_1,Expiry_month_1,Expiry_week_in_year_1,Expiry_week_in_year_2,Expiry_week_in_year_3,Expiry_week_in_year_4,Expiry_week_in_year_5,Expiry_week_in_year_6,Expiry_week_in_year_7,Expiry_week_in_year_8,Expiry_week_in_year_9
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0
1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0


## Merging the Numerical and Categorical features.

In [28]:
df_engineered_features = pd.concat([df_numerical_normalized.reset_index(drop=True),df_cat.reset_index(drop=True)], axis=1)
df_engineered_features.head()

,Income,Total Claim Amount,Customer Lifetime Value,Months Since Policy Inception,Months Since Last Claim,Number of Policies,Monthly Premium Auto,Number of Open Complaints,Sqrt_Total Claim Amount,Log_Customer Lifetime Value,Exponential_Number of Policies,Reciprocal_Monthly Premium Auto,Boxcox_Number of Open Complaints,Response,Education_1,Education_2,Education_3,Education_4,Policy Type_1,Policy Type_2,Sales Channel_1,Sales Channel_2,Sales Channel_3,State_1,State_2,State_3,State_4,Vehicle Class_1,Vehicle Class_2,Vehicle Class_3,Vehicle Class_4,Vehicle Class_5,EmploymentStatus_1,EmploymentStatus_2,EmploymentStatus_3,EmploymentStatus_4,Marital Status_1,Marital Status_2,Coverage_1,Coverage_2,Policy_1,Policy_2,Policy_3,Policy_4,Policy_5,Policy_6,Policy_7,Policy_8,Vehicle Size_1,Vehicle Size_2,Renew Offer Type_1,Renew Offer Type_2,Renew Offer Type_3,Location Code_1,Location Code_2,Gender_1,Expiry_month_1,Expiry_week_in_year_1,Expiry_week_in_year_2,Expiry_week_in_year_3,Expiry_week_in_year_4,Expiry_week_in_year_5,Expiry_week_in_year_6,Expiry_week_in_year_7,Expiry_week_in_year_8,Expiry_week_in_year_9
0,0.612827,-0.169640,-0.762878,-1.543287,1.678099,-0.822648,-0.703925,-0.42225,-0.004756,-1.262494,-1.033971,0.847769,0.691719,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0
1,-1.239617,2.400737,-0.149245,-0.217334,-0.208186,2.106160,0.022691,-0.42225,2.019287,0.156450,1.818841,-0.363565,-1.641026,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0
2,0.365710,0.455734,0.710636,-0.360680,0.288205,-0.404247,0.429596,-0.42225,0.599263,1.095824,-0.211410,-0.799200,0.596805,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0
3,-1.239617,0.329769,-0.052263,0.606907,0.288205,1.687759,0.371467,-0.42225,0.486441,0.296114,1.597885,-0.743946,-1.641026,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0
4,0.203390,-1.018843,-0.755575,-0.145661,-0.307465,-0.822648,-0.587666,-0.42225,-1.139971,-1.234941,-1.033971,0.598955,0.528534,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0


In [29]:
df_engineered_features.to_csv('Data/df_engineered_features.csv', index=False)

In [30]:
df_cat = df_cat.join(df_numerical_normalized['Response'])
df_cat.to_csv('Data/df_engineered_categorical.csv', index=False)

df_numerical_normalized.to_csv('Data/df_engineered_numerical.csv', index = False)
